In [3]:
import pandas as pd
import numpy as np

In [4]:
col_names = ['Text_tokens', 'Hashtags', 'Tweet_id', 'Present_media', 'Present_links', 'Present_domains', 'Tweet_type', 'Language', 'Timestamp',
        'User_id', 'Follower_count', 'Following_count', 'Is_verified', 'Account_creation_time',
        'User_id_engaging', 'Follower_count_engaging', 'Following_count_engaging', 'Is_verified_engaging', 'Account_creation_time_engaging',
        'Engagee_follows_engager', 'Reply_engagement_timestamp', 'Retweet_engagement_timestamp', 'Retweet_with_comment_engagement_timestamp', 'Like_engagement_timestamp']
def clean_col(x):
    if(pd.isna(x)):
        return x
    else:
        return x.replace('\t', '|')
    

def process_chunk_tsv(df, col_to_clean=['Text_tokens', 'Hashtags', 'Present_media', 'Present_links', 'Present_domains'], isVal=False):

    df.columns = col_names

    # Convert boolean to 1 / 0
    df['Is_verified'] = df['Is_verified'].apply(lambda  x: 1 if x else 0)
    df['Is_verified_engaging'] = df['Is_verified_engaging'].apply(lambda  x: 1 if x else 0)

    for col in col_to_clean:
        df[col] = df[col].apply(lambda x: clean_col(x))
        df[col] = df[col].fillna(0)
    return df

In [5]:
df_training = pd.read_csv('/home/andreafiandro/NAS/training.tsv', sep='\u0001', header=None, nrows=100000)

# Cumulative like (da sviluppare su interazioni)

In [125]:
df_training = process_chunk_tsv(df_training)
df_count_like = df_training[['Tweet_id', 'Like_engagement_timestamp']]
df_count_like = df_count_like[~df_count_like['Like_engagement_timestamp'].isnull()]
df_count_like.loc[:,'Like'] = np.repeat(1, df_count_like.shape[0])
df_count_like['Tweet_id'].unique().shape
df_count_like = df_count_like.reset_index(drop=True)
df_count_like['cumulative_like'] = df_count_like.groupby(['Tweet_id'])['Like'].cumsum()
df_count_like.head()

# Following / Follower ratio

In [5]:
df_training = process_chunk_tsv(df_training)
df_training.loc[:,"follow_ratio_engaging"] = df_training.loc[:,'Following_count_engaging'] / df_training.loc[:,'Follower_count_engaging']
df_training.head()

,Text_tokens,Hashtags,Tweet_id,Present_media,Present_links,Present_domains,Tweet_type,Language,Timestamp,User_id,...,User_id_engaging,Follower_count_engaging,Following_count_engaging,Is_verified_engaging,Account_creation_time_engaging,Engagee_follows_engager,Reply_engagement_timestamp,Retweet_engagement_timestamp,Retweet_with_comment_engagement_timestamp,Like_engagement_timestamp
0,101|6417|3410|3398|3184|1909|56910|16838|82904...,0,D4D1EBDE74F74C5DA529959AF979625C,0,0,0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,...,0000006C3074607050F1339DDCB890BB,27428,600,0,1520948869,False,NaN,NaN,NaN,NaN
1,101|14120|131|120|120|188|119|11170|120|178|71...,0,BFB529DAB6D384EB83E899A72AB3830D,Photo,0,0,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,...,000005C520010F8917EEAB6F5B6EC1C4,17,77,0,1569692352,True,NaN,NaN,NaN,NaN
2,101|62342|10858|54439|19571|22480|7831|8129|22...,0,519078C7834E9642508F72A6C2D0F3B7,0,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,...,000006829BEADA9EEA695CF0C334B426,2,29,0,1568107028,False,NaN,NaN,NaN,1.581247e+09
3,101|58955|10898|103305|1901|16181|7168|17984|1...,F0F2FBE57F08E7C4326682B5EDA63E3E|E78674D323461...,52AAE9E33EFAC8C478C57B31A9E31ED1,0,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,...,000006829BEADA9EEA695CF0C334B426,3,30,0,1568107028,False,NaN,NaN,NaN,NaN
4,101|2435|5656|2594|8279|8623|1925|64126|14725|...,0,89C1298C55EB3D68E2784F0BFB69E6F8,0,0,0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,...,000006829BEADA9EEA695CF0C334B426,3,30,0,1568107028,False,NaN,NaN,NaN,NaN


# Tempo passato da creazione a tweet

In [ ]:
df_training = process_chunk_tsv(df_training)
df_training['Elapsed_time_author'] = df_training['Timestamp'] - df_training['Account_creation_time']
df_training['Elapsed_time_user'] = df_training['Timestamp'] - df_training['Account_creation_time_engaging']
df_training.head()

# N° Like per utente

In [6]:
df_training = process_chunk_tsv(df_training)
df_count_like = df_training[['User_id_engaging', 'Like_engagement_timestamp']]
df_count_like = df_count_like[~df_count_like['Like_engagement_timestamp'].isnull()]
df_count_like.head()

,User_id_engaging,Like_engagement_timestamp
2,000006829BEADA9EEA695CF0C334B426,1.581247e+09
7,00000860E80C67D8C46CE57C64DE9444,1.581296e+09
9,00000860E80C67D8C46CE57C64DE9444,1.581009e+09
10,00000860E80C67D8C46CE57C64DE9444,1.581357e+09
13,0000094443AB46ED4A5C5E201EA3BCAE,1.581315e+09


In [ ]:
df_count_like['User_id_engaging'].unique().shape
df_count_like = df_count_like.reset_index(drop=True)
df_count_like['cumulative_like'] = df_count_like.groupby(['User_id_engaging'])['Like'].cumsum()
df_count_like.head()